# Responses API の MCP ツールの使用ガイド

エージェント型アプリケーションの構築では、外部サービスへの接続が必要になることがよくあります。従来は、関数呼び出しを通じて行われており、すべてのアクションがモデルからバックエンドへ、そして外部サービスへとラウンドトリップを行い、レスポンスを待ってから最終的に結果をモデルに返すという流れでした。このプロセスでは複数のネットワークホップと大幅な遅延が発生し、スケールや管理が煩雑になります。

Responses APIのホスト型Model Context Protocol（MCP）ツールは、これを簡単にします。各関数呼び出しを特定のサービスに手動で接続する代わりに、モデルを一度設定してMCPサーバー（または複数のサーバー！）を指すように構成できます。そのサーバーは集約型ツールホストとして機能し、「商品カタログを検索」や「カートにアイテムを追加」などの標準コマンドを公開します。これにより、ツールのオーケストレーションがシンプルになり、集約管理が可能になります。MCPを使用することで、モデルはMCPサーバーと直接やり取りし、遅延を削減してバックエンドの調整を不要にします。

## MCPツールによって簡素化されたユースケース

MCPは、外部サービスと連携する製品を構築する際の摩擦を大幅に削減し、異なるサービスをシームレスに結び付けることを可能にします。以下は、かつては摩擦を伴っていたものの、モデルがリモートMCPサーバーと直接通信できるようになったことで、現在ははるかに簡単になったユースケースのサンプルです。

| **ドメイン**                 | **MCPツールによって実現されたユースケース**                                                                                                                                                  | **以前の摩擦**                                                                                                      |
|---------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| **コマース / 決済**   | - Shopifyカートにアイテムを追加し、1回のやり取りでチェックアウトURLを返す — `"Allbirds Men's Tree Dasher 2のサイズ10を追加"` → カートリンク  <br> - Stripe決済リンクを生成 | 関数呼び出しでは、カスタムの`cart_add`や`create_payment_link`ラッパーを書き、独自のリレーサーバーをホストする必要がありました。 |
| **DevOps & コード品質**| - Sentryで特定のファイルの最新エラーを取得し、同じ会話内で修正案を含むGitHubイシューを開く                                                    | 1つのアシスタントループ内で2つの異なるサードパーティAPIを連鎖させるには、Webhookの接続と状態の管理が必要でした。                |
| **メッセージング / 通知** | - Web検索で朝のサッカーのトップヘッドラインを取得し、Twilioで要約を電話番号にテキスト送信する、これらを1回の呼び出しで実行                                                 | バックエンドで2つのツール呼び出しを繋ぎ合わせ、最終的なSMSペイロードを自分でバッチ処理する必要がありました。                             |

## ツールの動作原理

高レベルでは、MCPツールは以下のように動作します：

1. **サーバーの宣言**: `tools`配列にMCPブロックを追加すると、Responses APIランタイムは最初にサーバーが使用するトランスポートを検出します。これは新しい「ストリーマブルHTTP」または古いHTTP-over-SSE方式のいずれかで、そのプロトコルを使用してトラフィックを処理します。

2. **ツールリストのインポート**: ランタイムはサーバーの`tools/list`を呼び出し、あなたが提供するヘッダー（APIキー、OAuthトークンなど）を渡します。その後、結果をモデルのコンテキスト内の`mcp_list_tools`項目に書き込みます。この項目が存在する間は、リストが再取得されることはありません。`allowed_tools`を使用して、モデルが見ることができる内容を制限できます。

    OpenAIは各リクエスト後に、ヘッダー値とMCP `server_url`のスキーマ、ドメイン、サブドメイン以外のすべてを破棄します。認証キーとサーバーURLは、すべてのAPI呼び出しに含める必要があります。これらの値はレスポンスオブジェクトには表示されません。スキーマは可能な場合は「strict」モードを使用し、そうでなければそのまま読み込まれます。

3. **ツールの呼び出しと承認**: モデルが利用可能なアクションを認識すると、それを実行できます。各実行は`mcp_tool_call`項目を生成し、デフォルトではストリームがあなたの明示的な承認を待って一時停止しますが、サーバーを信頼できるようになったらこれを無効にできます。

    承認後、ランタイムは呼び出しを実行し、結果をストリームバックし、モデルは別のツールをチェーンするか最終的な回答を返すかを決定します。

## MCPを使った構築時のベストプラクティス

MCPはまだ初期段階にあるため、構築時にモデルのパフォーマンスと動作を改善できるベストプラクティスをご紹介します。

### ペイロードの肥大化を避けるためのツールフィルタリング

リモートサーバーは、モデルがそれらをどのように解釈し使用するかを考慮せずに、多数のツールを公開することがよくあります。デフォルトでは、これにより数十のエンドポイントが含まれる可能性があり、それぞれに名前、説明、JSONスキーマなどの詳細な定義が付随し、モデルのコンテキストに数百のトークンを追加してレイテンシを増加させます。これをさらに悪化させるのは、多くのサーバーがStripeの請求書レコード全体のような完全なデータオブジェクトを返すことです。これは、モデルのタスクに関連するのがわずか数フィールドだけであっても行われます。本番環境でのパフォーマンスを最適化するには、Responses APIの`allowed_tools`パラメータを使用して、サーバーの`mcp_list_tools`から含まれるツールを制限してください。これにより、トークンのオーバーヘッドが削減され、応答時間が改善され、モデルの決定空間が狭められます。また、書き込み操作が可能なツールや、財務的またはセキュリティ上の影響があるツールなど、特定のツールを完全に除外することも検討してください。

In [ ]:
curl https://api.openai.com/v1/responses -i \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4.1",
    "tools": [
        {
            "type": "mcp",
            "server_label": "gitmcp",
            "server_url": "https://gitmcp.io/openai/tiktoken",
            "allowed_tools": ["search_tiktoken_documentation", "fetch_tiktoken_documentation"],
            "require_approval": "never"
        }
    ],
    "input": "how does tiktoken work?"
}'

### キャッシュによるレイテンシとトークンの削減、および高複雑度タスクへの推論モデルの活用

モデルが初めてサーバーに接続する際、追加した各MCPサーバーに対して`mcp_list_tools`タイプの新しいアイテムが作成されます。このアイテムがモデルのコンテキストに存在する限り、サーバーに対して再度`tools/list`を呼び出すことはありません。これはユーザー会話レベルでのキャッシュに相当します。`mcp_list_tools`が存在しない場合、MCPサーバーからツールのリストを再度インポートします。後続のAPIリクエストで`previous_response_id`を渡すことは、フォローアップターンでモデルのコンテキストに`mcp_list_tools`アイテムが存在することを保証する一つの方法です。または、新しいレスポンスにアイテムを手動で渡すこともできます。レイテンシと出力トークン数に影響するもう一つの要因は、推論モデルを使用するかどうかです。推論モデルは推論トークンと同様に、はるかに多くの出力トークンを生成するためです。例として、推論モデルを使用した場合と使用しない場合で生成されるトークン数を比較する以下の2つのサンプルcurlを見てみましょう：

申し訳ございませんが、翻訳すべきテキストが「Scenario 1: non-reasoning model」の見出しのみのようです。

この部分を翻訳すると：

**シナリオ1：非推論モデル**

もし他にも翻訳が必要な内容がございましたら、お知らせください。

In [ ]:
curl https://api.openai.com/v1/responses \   
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4.1",
    "tools": [
      {
        "type": "mcp",
        "server_label": "gitmcp",
        "server_url": "https://gitmcp.io/openai/tiktoken",
        "require_approval": "never"
      }
    ],
    "input": "how does tiktoken work?" 
  }'

In [ ]:
  "usage": {
    "input_tokens": 280,
    "input_tokens_details": {
      "cached_tokens": 0
    },
    "output_tokens": 665,
    "output_tokens_details": {
      "reasoning_tokens": 0
    },
    "total_tokens": 945
  }

シナリオ2: `previous_response_id`なしの推論モデル

In [ ]:
curl https://api.openai.com/v1/responses \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "o4-mini",
    "tools": [
      {
        "type": "mcp",
        "server_label": "gitmcp",
        "server_url": "https://gitmcp.io/openai/tiktoken",
        "require_approval": "never"
      }
    ],
    "input": "how does tiktoken work?",
    "reasoning": {
      "effort": "medium",
      "summary": "auto"
    } 
  }'

In [ ]:
  "usage": {
    "input_tokens": 36436,
    "input_tokens_details": {
      "cached_tokens": 22964
    },
    "output_tokens": 1586,
    "output_tokens_details": {
      "reasoning_tokens": 576
    },
    "total_tokens": 38022 
    }

### 他のツールとMCPを使用する

MCPツールはツール配列の単なる別のエントリなので、モデルは`code_interpreter`、`web_search_preview`、`image_generation`などの他のホストされたツールや、あなたが定義するカスタムツールとシームレスに使用できます。また、複数のリモートMCPサーバーを一緒に使用することも可能です。

この例では、架空のヨガウェアストアの価格分析エージェントを作成します：まず、Alo Yoga MCPサーバーから女性用ショーツ、ヨガパンツ、タンクトップの現在の競合他社価格を取得し、次にホストされたweb-searchツールを介してUniqloから同じ3つのカテゴリの価格を取得します。Code Interpreterを使用して、Filesエンドポイントで事前に読み込まれたCSVから先週の売上を分析し、アイテムごとの収益と平均注文額を計算します。その後、新しく取得したUniqloとAlo Yogaのベンチマークと各アイテムの価格差を測定します。市場価格より15パーセント以上高いまたは低い価格の製品にはフラグが立てられ、エージェントは不一致と主要な収益統計を要約した簡潔なテキストレポートを提供します。

In [ ]:
system_prompt= """You are a pricing analyst for my clothing company. Please use the MCP tool 
to fetch prices from the Alo Yoga MCP server for the categories of women's 
shorts, yoga pants, and  tank tops. Use only the MCP server for Alo yoga data, don't search the web. 

Next, use the web search tool to search for Uniqlo prices for women's shorts, yoga pants, and tank tops. 

In each case for Alo Yoga and Uniqlo, extract the
price for the top result in each category. Also provide the full URLs
 
Using the uploaded CSV file of sales data from my store, and with the 
code interpreter tool calculate revenue by product item, compute average order-value on a transaction level, and calculate the percentage price gap between the CSV data and Uniqlo/Alo Yoga prices. 
Flag products priced 15% or more above or below the market. 
Create and output a short report including the findings.

# Steps

1. **Fetch Alo Yoga Prices:**
   - Use the Alo Yoga MCP server to fetch prices for the following products:
High-Waist Airlift Legging
Sway Bra Tank
 5" Airlift Energy Short

- Ensure you find prices for each. 
- Extract the price of the top result for each category.
- include  URL links 


2. **Query Uniqlo Prices:**
   - Use the Web-Search tool to search non-sale prices for the following Uniqlo products: 
Women's AIRism Soft Biker Shorts
Women's AIRism Soft Leggings
Women's AIRism Bra Sleeveless Top
- Ensure you find non-sale prices for each. 
- Extract the price for the top result in each category.
- include  URL links 

3. **Sales Data Analysis:**
   - Use the uploaded CSV sales data to calculate revenue across each 
   product item.
   - Determine the average order-value on a transaction level.
   - For each SKU, compute the percentage price gap between the 
   CSV data and Uniqlo/Alo Yoga prices.
   - Flag products priced ≥ 15% above or below the market.

4. **Report:**
   - Compile and output a report including the flagging results

# Output Format
- A short text report explaining:
  - Any products that are priced ≥ 15% above or below the market, 
  with specific details. """

以下は、上記のシステムプロンプトのプレースホルダーを含むサンプルのcurlです。

In [ ]:
curl https://api.openai.com/v1/responses \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4.1",
    "input": [
      {
        "role": "system",
        "content": [
          {
            "type": "input_text",
            "text": "ABOVE_SYSTEM_PROMPT"
          }
        ]
      }
    ],
    "tools": [
      {
        "type": "web_search_preview",
        "user_location": {
          "type": "approximate",
          "country": "US"
        },
        "search_context_size": "medium"
      },
      {
        "type": "code_interpreter",
        "container": {
          "type": "auto",
          "file_ids": [
            "file-WTiyGcZySaU6n218gj4XxR"
          ]
        }
      },
      {
        "type": "mcp",
        "server_url": "https://www.aloyoga.com/api/mcp",
        "server_label": "aloyoga",
        "allowed_tools": [
          "search_shop_catalog",
          "get_product_details"
        ],
        "require_approval": "never"
      }
    ],
    "temperature": 1,
    "max_output_tokens": 2048,
    "top_p": 1,
    "store": true
  }'


モデルは、MCPツールとウェブ検索の結果をコードインタープリターのステップに引き継ぎ、読みやすい形式でフォーマットされた以下の内容を含むレポートを作成することができます：

---
#### **価格比較と収益分析レポート**

**あなたの店舗の売上・価格分析**

- **商品別収益:**
    - ショーツ: $6,060
    - タンクトップ: $6,150
    - ヨガパンツ: $12,210
- **平均注文金額:** $872.14
- **あなたの店舗のカテゴリ別平均販売価格:**
    - ショーツ: $60.00
    - タンクトップ: $75.00
    - ヨガパンツ: $110.00

#### **市場との価格ギャップ**

| カテゴリ | 店舗平均価格 | vs Alo Yogaギャップ (%) | フラグ (≥15%) | vs Uniqloギャップ (%) | フラグ (≥15%) |
| --- | --- | --- | --- | --- | --- |
| ショーツ | $60.00 | -31.8% | **YES** | +100.7% | **YES** |
| タンクトップ | $75.00 | -14.8% |  | +114.9% | **YES** |
| ヨガパンツ | $110.00 | -14.1% |  | +267.9% | **YES** |

#### **推奨事項とフラグ**

**フラグ対象商品（≥15%の価格ギャップ）:**

- **ショーツ:** Alo Yogaより31.8%安いが、Uniqloより100.7%高い。
- **タンクトップ:** Uniqloより114.9%以上高い。
- **ヨガパンツ:** Uniqloより267.9%高い。

ショーツは、プレミアム競合他社（Alo Yoga）よりも大幅に安く価格設定されていますが、バジェット代替品（Uniqlo）よりもはるかに高くなっています。プレミアムセグメントで競争したい場合は、価格を上げることを検討してください。バジェット購入者をターゲットにしたい場合は、価格を下げることが正当化できるかもしれません。タンクトップとヨガパンツの大部分も同様のポジションにあります—Aloよりもはるかに安いが、Uniqloよりもかなり高い価格設定です。

___

### MCPツール呼び出しを改善するためのプロンプトガイドライン

使用ケースによっては、特にcatalog-searchツールを使用する際に、モデルが多くのMCP呼び出しを実行することがあります。大規模な商品インベントリに対する無限の反復を避けるために、モデルに検索をN個のアイテムに制限し、ユーザーが明示的により多くの情報を求めた場合にのみ継続を提案するよう指示することが有効です。これにより、応答が焦点を絞った簡潔なものになります。

使用しているMCPサーバーに包括的な`mcp_list_tools`が含まれている場合、モデルに正しいサーバーを選択する方法と、冗長な呼び出しを発行する代わりに必要な情報を取得したら停止する方法を示すために、対象を絞ったfew-shotの例をいくつか含めることも価値があります。

最後に、ユーザークエリから重要な情報（サイズ、色、製品ライン等）が欠けている場合、広範囲な検索を開始するのではなく、フォローアップの質問をするべきであることをモデルに思い出させるガイダンスを追加します。この小さなプロンプトの後押しにより、不要なツール呼び出しが減り、回答の品質が向上します。以下は、これらのガイドラインがどのように組み合わされるかを示すサンプルプロンプトです：

In [ ]:
system_prompt= """You are an AI assistant that can call the following MCP servers:

1. allbirds_store
2. gitmcp

Steps
1. Use the MCP servers above to answer the user query, not every MCP server will relevant for a given query so you can choose which ones to invoke.  
2. If the user’s request lacks essential details (size, color, etc.), ask a follow-up question first rather than guessing and using the MCP server tool prematurely. 
3. If searching for products in a catalog, only return 4 results at a time before asking the user whether they want to see more. 

### Few-shot examples

# EX1 — Direct product search, then fetch variant details
user: Do you have the Allbirds Tree Runner in men’s size 10?  
assistant: {"name":"allbirds_store.search_shop_catalog",
            "arguments":{"query":"Tree Runner",
                         "context":"Men’s size 10 shoes"}}
# (assume the search returns product_id "gid://shopify/Product/987")  
assistant: {"name":"allbirds_store.get_product_details",
            "arguments":{"product_id":"gid://shopify/Product/987",
                         "options":{"Size":"10"}}}

# EX2 — Clarify missing size, then two-step color variant lookup
user: I want the Tree Runner in blue.  
assistant: Sure — what size do you need?  
user: Size 10.  
assistant: {"name":"allbirds_store.search_shop_catalog",
            "arguments":{"query":"Tree Runner",
                         "context":"Blue variant, size 10"}}
# (assume the search returns product_id "gid://shopify/Product/987")  
assistant: {"name":"allbirds_store.get_product_details",
            "arguments":{"product_id":"gid://shopify/Product/987",
                         "options":{"Size":"10","Color":"Blue"}}}

# EX3 — Git docs: search then fetch specific file
user: Can you show me how tiktoken does byte-pair encoding?  
assistant: {"name":"gitmcp.search_website_documentation",
            "arguments":{"query":"tiktoken byte-pair encoding"}}
# (assume the search returns document_id "docs/ENCODING.md")  
assistant: {"name":"gitmcp.fetch_website_documentation",
            "arguments":{"document_id":"docs/ENCODING.md"}} """


In [ ]:
curl https://api.openai.com/v1/responses \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4.1",
    "input": [
      {
        "role": "system",
        "content": [
          {
            "type": "input_text",
            "text": "ABOVE_SYSTEM_PROMPT"
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "input_text",
            "text": "find me womens tree loungers in size 8"
          }
        ]
      }
    ],
    "tools": [
      {
        "type": "mcp",
        "server_url": "https://www.allbirds.com/api/mcp",
        "server_label": "allbirds",
        "allowed_tools": [
          "search_shop_catalog",
          "get_cart",
          "update_cart",
          "search_shop_policies_and_faqs",
          "get_product_details"
        ],
        "require_approval": "never"
      },
      {
        "type": "mcp",
        "server_label": "gitmcp",
        "server_url": "https://gitmcp.io/openai/tiktoken",
        "allowed_tools": [
          "fetch_tiktoken_documentation",
          "search_tiktoken_documentation",
          "search_tiktoken_code",
          "fetch_generic_url_content"
        ],
        "require_approval": "never"
      }
    ],
    "temperature": 1,
    "max_output_tokens": 2048
  }'


## 結論

Responses APIにおけるホスト型MCPツールは、外部サービスへのアクセスを特注の配管作業から、APIのファーストクラス機能へと変えます。リモートサーバーに接続し、ランタイムにツールリストをキャッシュさせ、`allowed_tools`でそのリストを絞り込むことで、余分なネットワークホップを排除し、トークンのオーバーヘッドを削減し、モデルに簡潔で発見可能なアクションセットを提供します。`code_interpreter`、`web_search_preview`、`image_gen`などの組み込みツールと組み合わせることで、MCPは売上データの分析、本番エラーのトリアージ、チェックアウトフローの自動化など、豊富なマルチサービスワークフローを実現します。